In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pointCollection as pc
import scipy.sparse as sp
from LSsurf import RDE

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib widget

In [73]:
def read_xovers(file, fields=['x','y','delta_time','h_li','h_li_sigma','spot', 'rgt', 'dh_fit_dx','dh_fit_dy','atl06_quality_summary']):

    m = pc.data().from_h5(file, field_dict={None:['grounded','x','y','slope_x','slope_y']})
    v=pc.data(columns=2)
    data = [pc.data().from_h5(file, field_dict={group:fields+['W']}) for group in ['data_0','data_1']]
    
    d=pc.data()
    for field in fields:
        if field=='W':
            continue
        temp = np.zeros((m.size, 2))
        for col in [0, 1]:
            temp[:, col] = np.sum(data[col].W * getattr(data[col], field), axis=1)
        
        if field in ['rgt','cycle','spot']:
            # integer fields:
            v.assign({ field : np.round(temp).astype(int)})
        else:
            v.assign({ field : temp})
            
        d.assign({ field : np.diff(temp, axis=1)})
    for item in [v, d]:
        item.__update_size_and_shape__()
    
    return v, d, m, data



In [74]:
v, d, m, D = read_xovers('/Users/ben/Data/xovers/v005/cxx/c08/E200000_N0.h5')

In [76]:
v.columns

2

In [78]:
def calc_biases(d, v, ind):
    v1=v[ind]
    d1=d[ind]
    m1=m[ind]

    uS, iS=np.unique(v1.spot, return_inverse=True)
    uT, iT=np.unique(v1.rgt,  return_inverse=True)
    row=np.arange(v1.shape[0])
    G_sp=sp.coo_matrix((np.c_[-np.ones(v1.shape[0]), np.ones(v1.shape[0])].ravel(),(np.c_[row, row].ravel(), iS.ravel())))
    G_rgt=sp.coo_matrix((np.c_[-np.ones(v1.shape[0]), np.ones(v1.shape[0])].ravel(),(np.c_[row, row].ravel(), iT.ravel())))
    G_sp_mean=sp.coo_matrix((np.r_[np.ones(len(uS)), np.zeros(len(uT))],
                              (np.zeros(len(uS)+len(uT), dtype=int), 
                                np.arange(len(uS)+len(uT), dtype=int)))) 
    G_rgt_mean=sp.coo_matrix((np.r_[np.zeros(len(uS)), np.ones(len(uT))],
                              (np.zeros(len(uS)+len(uT), dtype=int), 
                                np.arange(len(uS)+len(uT), dtype=int)))) 


    G=sp.vstack([sp.hstack([G_sp, G_rgt]), G_sp_mean, G_rgt_mean, 0.0001*sp.eye(len(uS)+len(uT))])
    G=G.tocsr()
    data=np.r_[d1.h_li.ravel(), np.zeros(G.shape[0]-d1.shape[0])]
    r=np.zeros(d1.shape[0])
    good=np.ones(G.shape[0], dtype=bool)
    last_good=np.zeros_like(good)
    sigma=1.e4
    n_data=d1.shape[0]
    while np.sum(np.logical_xor(good,last_good))>0:
        ii=np.flatnonzero(good)
        mm=sp.linalg.spsolve(G[ii,:].T.dot(G[ii,:]), G[ii,:].T.dot(data[ii]))
        r=d1.h_li.ravel()-G[0:d1.size,:].dot(mm)
        sigma=RDE(np.abs(r[ii[ii<n_data]]))
        last_good=good
        good=np.ones(G.shape[0], dtype=bool)
        good[np.argwhere(np.abs(r)>3*sigma)]=False
    b_spot=np.zeros(6)+np.NaN
    for ind,spot in enumerate(uS):
        b_spot[int(spot)-1]=mm[ind]
    b_rgt = np.zeros(1387)+np.NaN
    for ind, rgt in enumerate(uT):
        b_rgt[int(rgt)-1]=mm[len(uS)+ind]
    #tbar=np.mean(v1.delta_time)
    return b_spot, b_rgt, sigma

In [80]:
dt=10*24*3600
t_vals=np.arange(np.min(v.delta_time)+dt/2, np.max(v.delta_time), dt/2)
b_spot=np.zeros((len(t_vals), 6))+np.NaN
b_rgt=np.zeros((len(t_vals), 1387))+np.NaN
sigma=np.zeros(len(t_vals))+np.NaN
tbar=np.zeros(len(t_vals))+np.NaN
for it, ti in enumerate(t_vals):
    ii = np.all(np.abs(v.delta_time-ti)<dt/2, axis=1)
    if np.sum(ii)>10:
        b_spot[it,:], b_rgt[it,:], sigma[it]=calc_biases(d, v, np.argwhere(ii))

In [77]:
ind=ii
if True:
    v1=v[ind]
    d1=d[ind]
    m1=m[ind]

    uS, iS=np.unique(v1.spot, return_inverse=True)
    uT, iT=np.unique(v1.rgt,  return_inverse=True)
    row=np.arange(v1.shape[0])
    G_sp=sp.coo_matrix((np.c_[-np.ones(v1.shape[0]), np.ones(v1.shape[0])].ravel(),(np.c_[row, row].ravel(), iS.ravel())))


In [81]:
plt.figure(); plt.plot(b_spot)
plt.legend(['spot '+str(ii) for ii in range(1, 7)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
plt.clf(); plt.hist(np.nanmax(b_rgt, axis=0)-np.nanmin(b_rgt, axis=0), 20);
plt.figure(); plt.hist(np.nanmean(b_rgt, axis=0), 20);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.figure(); 
plt.scatter(v.x, v.y, 2, c=v.delta_time/24/3600/365.25+2018, cmap='jet'); 
plt.colorbar()

In [ ]:
good=np.all(np.abs(v.x+1j*v.y)<1.e8, axis=1)
np.sum(good)

In [ ]:
data[0].W

In [ ]:
v